## Exploring the Twitter Emotion Detection dataset using Autolabel

#### Set up the API keys for the providers you want to use

In [ ]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [ ]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [1]:
from autolabel import get_data

get_data('twitter_emotion_detection')

2023-06-28 15:10:47 autolabel.utils ERROR: twitter_emotion_detection not in list of available datasets: ['banking', 'civil_comments', 'ledgar', 'walmart_amazon', 'company', 'squad_v2', 'sciq', 'conll2003', 'movie_reviews']. Exiting...


This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [2]:
from autolabel import LabelingAgent

In [3]:
config = {
    "task_name": "EmotionClassification",
    "task_type": "multilabel_classification",
    "dataset": {"label_column": "labels", "label_separator": ", ", "delimiter": ","},
    "model": {"provider": "openai", "name": "gpt-3.5-turbo"},
    "prompt": {
        "task_guidelines": "You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.\nYour job is to correctly label the provided input example into one or more of the following categories:\n{labels}",
        "output_guidelines": 'You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"',
        "labels": [
            "neutral",
            "anger",
            "anticipation",
            "disgust",
            "fear",
            "joy",
            "love",
            "optimism",
            "pessimism",
            "sadness",
            "surprise",
            "trust",
        ],
        "few_shot_examples": "seed.csv",
        "few_shot_selection": "semantic_similarity",
        "few_shot_num": 5,
        "example_template": "Input: {example}\nOutput: {labels}",
    },
}

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `multilabel_classification` (since it's a multi label classification task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.labels`: `['neutral', 'anger', 'anticipation', 'disgust', 'fear', ...]` (the full list of labels to choose from)
* `prompt.task_guidelines`: `'You are an expert at classifying tweets as...` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 5 (how many labeled examples to provide to the LLM)

In [4]:
config

{'task_name': 'EmotionClassification',
 'task_type': 'multilabel_classification',
 'dataset': {'label_column': 'labels',
  'label_separator': ', ',
  'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.\nYour job is to correctly label the provided input example into one or more of the following categories:\n{labels}',
  'output_guidelines': 'You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"',
  'labels': ['neutral',
   'anger',
   'anticipation',
   'disgust',
   'fear',
   'joy',
   'love',
   'optimism',
   'pessimism',
   'sadness',
   'surprise',
   'trust'],
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 5,
  'example_template': 'Input: {example}\nOutput: 

In [5]:
# create an aggent for labeling
agent = LabelingAgent(config=config)

In [6]:
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.9705 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: anger, disgust, fear

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: fear

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [7]:
# now, do the actual labeling
ds = agent.run(ds, max_items=100)

2023-06-28 15:11:19 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='148569753' created_at=datetime.datetime(2023, 6, 28, 15, 10, 
34, 979450) task_id='f3f9865e0ef223de46a41a00d30b0847' dataset_id='6972c71134baa2a223a510daf02a6997' 
current_index=0 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

/Users/workrefuel/.pyenv/versions/3.8.16/envs/refuel/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['anxiety'] will be ignored
  warnings.warn(


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.1778 │ 5       │ 0.0      │ 1.0             │
└────────┴─────────┴──────────┴─────────────────┘

...

/Users/workrefuel/.pyenv/versions/3.8.16/envs/refuel/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:89
5: UserWarning: unknown class(es) ['anxiety', 'happiness', 'trauma'] will be ignored
  warnings.warn(

Actual Cost: 0.0025


/Users/workrefuel/.pyenv/versions/3.8.16/envs/refuel/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['anxiety', 'happiness', 'trauma'] will be ignored
  warnings.warn(


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.4507 │ 100     │ 0.08     │ 1.0             │
└────────┴─────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at a 0.4507 macro f1 when labeling the first 100 examples. Let's see if we can use confidence scores to improve f1 further by removing the less confident examples from our labeled set.

### Compute confidence scores

In [8]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

In [9]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [10]:
agent = LabelingAgent(config=config)

In [11]:
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.9705 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: anger, disgust, fear

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: fear

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
ds = agent.run(ds, max_items=100)

2023-06-28 15:12:59 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='2545097544' created_at=datetime.datetime(2023, 6, 28, 14, 36,
8, 629024) task_id='f1cc75949ce09a9b60ccfcd75bf6f7a4' dataset_id='6972c71134baa2a223a510daf02a6997' 
current_index=100 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

Metric: auroc: 0.7608695652173914


/Users/workrefuel/.pyenv/versions/3.8.16/envs/refuel/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['anxiety', 'happiness', 'trauma'] will be ignored
  warnings.warn(


...

/Users/workrefuel/.pyenv/versions/3.8.16/envs/refuel/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:89
5: UserWarning: unknown class(es) ['anxiety', 'happiness', 'trauma'] will be ignored
  warnings.warn(

Metric: auroc: 0.7609
Actual Cost: 0.0025


/Users/workrefuel/.pyenv/versions/3.8.16/envs/refuel/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['anxiety', 'happiness', 'trauma'] will be ignored
  warnings.warn(


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃ threshold ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 0.0833 │ 1       │ 0.0      │ 0.01            │ 0.9555    │
│ 0.25   │ 2       │ 0.5      │ 0.02            │ 0.9248    │
│ 0.4048 │ 7       │ 0.1429   │ 0.07            │ 0.8177    │
│ 0.4074 │ 8       │ 0.25     │ 0.08            │ 0.8099    │
│ 0.4091 │ 10      │ 0.2      │ 0.1             │ 0.7784    │
│ 0.4119 │ 11      │ 0.2727   │ 0.11            │ 0.7746    │
│ 0.3937 │ 12      │ 0.25     │ 0.12            │ 0.765     │
│ 0.3997 │ 13      │ 0.3077   │ 0.13            │ 0.7606    │
│ 0.4279 │ 29      │ 0.1379   │ 0.29            │ 0.6927    │
│ 0.4341 │ 30      │ 0.1667   │ 0.3             │ 0.6805    │
│ 0.4508 │ 35      │ 0.1429   │ 0.35            │ 0.6213    │
│ 0.4537 │ 36      │ 0.1667   │ 0.36            │ 0.6114    │
│ 0.4842 │ 49      │ 0.1224   │ 0.49            │ 0.5561    │
│ 0.4932 │ 50      │ 0.14     │ 0.5             │ 0.5553    │
│ 0.4629 │ 61      │ 0.1148   │ 0.61            │ 0.5011    │
│ 0.4639 │ 62      │ 0.129    │ 0.62            │ 0.4988    │
│ 0.4507 │ 100     │ 0.08     │ 1.0             │ 0.0206    │
└────────┴─────────┴──────────┴─────────────────┴───────────┘

Total number of failures: 0